In [1]:
import os, json
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [95]:
path_to_json = 'E:/growbydata/Growbydata/task_4/amazoncpi/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
final_json_to_csv_df = pd.DataFrame()
for file_name in (json_files):
    data = json.load(open(f"./amazoncpi/json/{file_name}"))
    df = pd.json_normalize(data["data"])
    # for header
    name = df[(df['depth']==16) & (df['element']=='H5')]['text'].reset_index(drop=True)
    #print(name)
    rating = df[(df['depth']==16)& (df['element']=='SPAN') & (df['height']==16) & (df['x']==716.40625)]['text'].reset_index(drop=True)
    #print(rating)
    review = df[(df['depth']==17) & (df['element']=='I') & (df['height']==18)]['attributes.class'].reset_index(drop=True)
    #print(review)
    price = df[(df['depth']==21) & (df['element']=='SPAN') & (df['height']==25) & (df['x']==633)]['text'].unique()
    price = pd.Series(price)
    #print(price)
    sold_by = df[(df['depth']==17) & (df['element']=='DIV') & (df['height']==20) & (df['width']==397) | (df['width']==382)]['text'].reset_index(drop=True)
    #print(sold_by)
    header_df = pd.concat([name, rating, review, price, sold_by],  axis=1)
    header_df.columns = ['name','raring','review','price','sold_by']
    #print(header_df)
    #header_df.to_csv('Header.csv')
    
    
    
    # for body
    sold_by = df[(df['depth']==14) & (df['element']=='DIV') & (df['height']==20) & (df['x']==633)]['text'].reset_index(drop=True)
    #print(sold_by)
    price = df[(df['depth']==17) & (df['element']=='SPAN') & (df['height']==25) & (df['width']==79.046875)]['text'].reset_index(drop=True)
    #print(price)
    review = df[(df['depth']==16) & (df['element']=='I') & (df['height']==12)]['attributes.class'].reset_index(drop=True)
    #print(review)
    rcount = df[(df['depth']==15) & (df['element']=='DIV') & (df['height']==40) & (df['x']==633)]['text'].reset_index(drop=True)
    rcount = rcount[~rcount.str.contains("FREE")] # drop row which conain free text
    rcount = rcount[~rcount.str.contains("19.95")]# drop row which conain free 19.95
    #print(rcount)
    rating_count = rcount.astype(str).str.split(' | ratings',expand=True)[0].str.strip("()").reset_index(drop=True)
    #print(rating_count)
    description = rcount.astype(str).str.split(')',expand=True)[1].reset_index(drop=True)
   # print(description)
    body_df = pd.concat([sold_by, price, review, rating_count, description],  axis=1)
    print(body_df)
    
    
    

                 text     text  \
0  Consumer Supply Co  $195.00   
1             AVLGear      NaN   

                                    attributes.class     0  \
0  a-icon a-icon-star-mini a-star-mini-4-5 aod-se...   538   
1  a-icon a-icon-star-mini a-star-mini-4-5 aod-se...  4281   

                                  1  
0  87% positive over last 12 months  
1  84% positive over last 12 months  


In [113]:
import pandas as pd
import os
import json
path_to_json = 'E:/growbydata/Growbydata/task_4/amazoncpi/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
final_json_to_csv_df = pd.DataFrame()
from_file = 0
index_file_mapping={}

for file_name in (json_files):
    #print(file_name)
    data = json.load(open(f"./amazoncpi/json/{file_name}"))
    df = pd.json_normalize(data["data"])
    df['file_from'] = file_name
    #print(df)
    # for body data
    category = ['category1', 'category2']
    
    from_file = from_file + 1
    for l in range(len(category)):
        if (l == 0):
            df = pd.json_normalize(data["data"])
            # for header
            name = df[(df['depth']==16) & (df['element']=='H5')]['text'].reset_index(drop=True)
            #print(name)
            rating = df[(df['depth']==16)& (df['element']=='SPAN') & (df['height']==16) & (df['x']==716.40625)]['text'].reset_index(drop=True)
            #print(rating)
            review = df[(df['depth']==17) & (df['element']=='I') & (df['height']==18)]['attributes.class'].reset_index(drop=True)
            #print(review)
            price = df[(df['depth']==21) & (df['element']=='SPAN') & (df['height']==25) & (df['x']==633)]['text'].unique()
            price = pd.Series(price)
            #print(price)
            sold_by = df[(df['depth']==17) & (df['element']=='DIV') & (df['height']==20) & (df['width']==397) | (df['width']==382)]['text'].reset_index(drop=True)
            #print(sold_by)
            final_dfs = pd.concat([name, rating, review, price, sold_by],  axis=1)
            final_dfs.columns = ['category1_name','category1_rating','category1_review','category1_price','category1_sold_by']
            #print(header_df)

            indexx = []
            field = []
            value = []
            j=1
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    indexx.append(f" {from_file}.{l+1}.{j}.{i+1}")
                j=j+1
            #print(index)
            file_name = file_name.split('.') # for file number
            file_name=file_name[0]
            index_file_mapping[file_name] = from_file   # for mapping 


            j=0        
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    field.append(content.index[i])
                    value.append(content[i])
                    j = j+1

            #from_file = from_file + 1
            # series
            s1 = pd.Series(indexx) 
            s2 = pd.Series(field)
            s3 = pd.Series(value)
            final_df =pd.concat([s1, s2, s3], axis=1)
            final_df.columns = ['index','field','value']
            final_df = final_df.sort_values(by=['field','index'])
            #print(final_df)
            final_json_to_csv_df = pd.concat([final_json_to_csv_df, final_df])
            #print(final_json_to_csv_df)

        # for body
        else: 
            sold_by = df[(df['depth']==14) & (df['element']=='DIV') & (df['height']==20) & (df['x']==633)]['text'].reset_index(drop=True)
            #print(sold_by)
            price = df[(df['depth']==17) & (df['element']=='SPAN') & (df['height']==25) & (df['width']==79.046875)]['text'].reset_index(drop=True)
            #print(price)
            review = df[(df['depth']==16) & (df['element']=='I') & (df['height']==12)]['attributes.class'].reset_index(drop=True)
            #print(review)
            rcount = df[(df['depth']==15) & (df['element']=='DIV') & (df['height']==40) & (df['x']==633)]['text'].reset_index(drop=True)
            rcount = rcount[~rcount.str.contains("FREE")] # drop row which conain free text
            rcount = rcount[~rcount.str.contains("19.95")]# drop row which conain free 19.95
            #print(rcount)
            rating_count = rcount.astype(str).str.split(' | ratings',expand=True)[0].str.strip("()").reset_index(drop=True)
            #print(rating_count)
            description = rcount.astype(str).str.split(')',expand=True)[1].reset_index(drop=True)
           # print(description)
            final_dfs = pd.concat([sold_by, price, review, rating_count, description],  axis=1)
            final_dfs.columns = ['category2_name','category2_price','category2_review','category2_rating_count','category2_description']
            #print(final_dfs)

            indexx = []
            field = []
            value = []
            j=1
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    indexx.append(f" {from_file}.{l+1}.{j}.{i+1}")
                j=j+1
            #print(index)
            file_name = file_name.split('.') # for file number
            file_name=file_name[0]
            index_file_mapping[file_name] = from_file   # for mapping 


            j=0        
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    field.append(content.index[i])
                    value.append(content[i])
                    j = j+1

            #from_file = from_file + 1
            # series
            s1 = pd.Series(indexx) 
            s2 = pd.Series(field)
            s3 = pd.Series(value)
            final_df =pd.concat([s1, s2, s3], axis=1)
            final_df.columns = ['index','field','value']
            final_df = final_df.sort_values(by=['field','index'])
            #print(final_df)
            final_json_to_csv_df = pd.concat([final_json_to_csv_df, final_df])
#final_json_to_csv_df.fillna('Nan')
print(final_json_to_csv_df)
#final_json_to_csv_df.to_csv("final_task444.csv",index=False)
index_file_mapping_df = pd.DataFrame(list(index_file_mapping.items()), columns = ['file_name','Indexing'])
#print(index_file_mapping_df)
#index_file_mapping_df.to_csv('map_index_t4.csv',index=False)

        index                   field  \
0     1.1.1.1          category1_name   
45   1.1.10.1          category1_name   
50   1.1.11.1          category1_name   
55   1.1.12.1          category1_name   
60   1.1.13.1          category1_name   
..        ...                     ...   
6     2.2.2.2         category2_price   
3     2.2.1.4  category2_rating_count   
8     2.2.2.4  category2_rating_count   
2     2.2.1.3        category2_review   
7     2.2.2.3        category2_review   

                                                value  
0   CAD Audio, 2 GXLD2HBAH Digital Wireless Combo ...  
45                                                NaN  
50                                                NaN  
55                                                NaN  
60                                                NaN  
..                                                ...  
6                                                 NaN  
3                                                 538  
8  

In [151]:
import pandas as pd
import os
import json
path_to_json = 'E:/growbydata/Growbydata/task_4/amazoncpi/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
final_json_to_csv_df = pd.DataFrame()
from_file = 0
index_file_mapping={}

for file_name in (json_files):
    #print(file_name)
    data = json.load(open(f"./amazoncpi/json/{file_name}"))
    df = pd.json_normalize(data["data"])
    df['file_from'] = file_name
    #print(df)
    # for body data
    category = ['category1', 'category2']
    from_file = from_file + 1
    for l in range(len(category)):
        if (l == 0):
            
            # for header
            name = df[(df['depth']==16) & (df['element']=='H5')]['text'].reset_index(drop=True)
            #print(name)
            rating = df[(df['depth']==16)& (df['element']=='SPAN') & (df['height']==16) & (df['x']==716.40625)]['text'].reset_index(drop=True)
            #print(rating)
            review = df[(df['depth']==17) & (df['element']=='I') & (df['height']==18)]['attributes.class'].reset_index(drop=True)
            #print(review)
            price = df[(df['depth']==21) & (df['element']=='SPAN') & (df['height']==25) & (df['x']==633)]['text'].unique()
            price = pd.Series(price)
            #print(price)
            #sold_by = df[(df['depth']==17) & (df['element']=='DIV') & (df['height']==20) & (df['width']==397) | (df['width']==382)]['text'].reset_index(drop=True)
            #print(sold_by)
            #final_dfs = pd.concat([name, rating, review, price, sold_by],  axis=1)
            #final_dfs.columns = ['category1_name','category1_rating','category1_review','category1_price','category1_sold_by']
            #print(final_dfs['category1_sold_by'])## error
            final_dfs = pd.concat([name, rating, review, price],  axis=1)
            final_dfs.columns = ['category1_name','category1_rating','category1_review','category1_price']

            indexx = []
            field = []
            value = []
            j=1
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    indexx.append(f" {from_file}.{l+1}.{j}.{i+1}")
                j=j+1
            #print(index)
            file_name = file_name.split('.') # for file number
            file_name=file_name[0]
            index_file_mapping[file_name] = from_file   # for mapping 


            j=0        
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    field.append(content.index[i])
                    value.append(content[i])
                    j = j+1

            #from_file = from_file + 1
            # series
            s1 = pd.Series(indexx) 
            s2 = pd.Series(field)
            s3 = pd.Series(value)
            final_df =pd.concat([s1, s2, s3], axis=1)
            final_df.columns = ['index','field','value']
            final_df = final_df.sort_values(by=['field','index'])
            #print(final_df)
            final_json_to_csv_df = pd.concat([final_json_to_csv_df, final_df])
            #print(final_json_to_csv_df)

        # for body
        else: 
            sold_by = df[(df['depth']==14) & (df['element']=='DIV') & (df['height']==20) & (df['x']==633)]['text'].reset_index(drop=True)
            #print(sold_by)
            price = df[(df['depth']==17) & (df['element']=='SPAN') & (df['height']==25) & (df['width']==79.046875)]['text'].reset_index(drop=True)
            #print(price)
            review = df[(df['depth']==16) & (df['element']=='I') & (df['height']==12)]['attributes.class'].reset_index(drop=True)
            #print(review)
            rcount = df[(df['depth']==15) & (df['element']=='DIV') & (df['height']==40) & (df['x']==633)]['text'].reset_index(drop=True)
            rcount = rcount[~rcount.str.contains("FREE")] # drop row which conain free text
            rcount = rcount[~rcount.str.contains("19.95")]# drop row which conain free 19.95
            #print(rcount)
            rating_count = rcount.astype(str).str.split(' | ratings',expand=True)[0].str.strip("()").reset_index(drop=True)
            #print(rating_count)
            description = rcount.astype(str).str.split(')',expand=True)[1].reset_index(drop=True)
           # print(description)
            final_dfs = pd.concat([sold_by, price, review, rating_count, description],  axis=1)
            final_dfs.columns = ['category2_name','category2_price','category2_review','category2_rating_count','category2_description']
            #print(final_dfs)

            indexx = []
            field = []
            value = []
            j=1
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    indexx.append(f" {from_file}.{l+1}.{j}.{i+1}")
                j=j+1
            #print(index)
            file_name = file_name.split('.') # for file number
            file_name=file_name[0]
            index_file_mapping[file_name] = from_file   # for mapping 


            j=0        
            for index,content in final_dfs.iterrows():
                for i in range(len(content)):
                    field.append(content.index[i])
                    value.append(content[i])
                    j = j+1

            #from_file = from_file + 1
            # series
            s1 = pd.Series(indexx) 
            s2 = pd.Series(field)
            s3 = pd.Series(value)
            final_df =pd.concat([s1, s2, s3], axis=1)
            final_df.columns = ['index','field','value']
            final_df = final_df.sort_values(by=['field','index'])
            #print(final_df)
            final_json_to_csv_df = pd.concat([final_json_to_csv_df, final_df])
final_json_to_csv_df = final_json_to_csv_df.fillna('Nan')
#print(final_json_to_csv_df)
final_json_to_csv_df.to_csv("final_task14.csv",index=False)
#index_file_mapping_df = pd.DataFrame(list(index_file_mapping.items()), columns = ['file_name','Indexing'])
#print(index_file_mapping_df)
#index_file_mapping_df.to_csv('map_index_t4.csv',index=False)